## Simple Gen AI APP Using Langchain

In [ ]:
# Code to set up environment variables for LangSmith tracking and import necessary libraries
import os
from dotenv import load_dotenv
load_dotenv()
## LangSmith Tracking
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY", "")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")

In [4]:
## Data Ingestion-- from the website we need to scrape the data
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/")
loader

In [5]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications.', 'language': 'en'}, page_content="\n\n\n\n\nGet started with LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\n\n\n\nSkip to main contentOur Building Ambient Agents with LangGraph course is now available on LangChain Academy!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceGet StartedOn this pageGet started with LangSmith\nLangSmith is a platform for building production-grade LLM applications.\nIt allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.\nObservabilityAnalyze tr

In [6]:
## Load Data --> Docs --> Divide our text into chunks --> text --> vectors --> Vector Embedding --> Vector StorE DB
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents=text_splitter.split_documents(docs)
docs = text_splitter.split_documents(docs)

In [7]:
documents

[Document(metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications.', 'language': 'en'}, page_content='Get started with LangSmith | 🦜️🛠️ LangSmith'),
 Document(metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications.', 'language': 'en'}, page_content='Skip to main contentOur Building Ambient Agents with LangGraph course is now available on LangChain Academy!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceGet StartedOn this pageGet started 

In [8]:
from langchain_community.embeddings import OllamaEmbeddings
embeddings = OllamaEmbeddings(model="llama2")

C:\Users\vidya\AppData\Local\Temp\ipykernel_19644\964864206.py:2: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="llama2")


In [9]:
from langchain_community.vectorstores import FAISS
vectorstoredb = FAISS.from_documents(documents, embeddings)

In [10]:
vectorstoredb

In [11]:
3#Query From a vector db
query = "Observability is important for any software application, but especially so for LLM applications"
result=vectorstoredb.similarity_search(query)
result[0].page_content

'Get started by adding tracing to your application.\nCreate dashboards to view key metrics like RPS, error rates and costs.\n\nEvals\u200b\nThe quality and development speed of AI applications depends on high-quality evaluation datasets and metrics to test and optimize your applications on. The LangSmith SDK and UI make building and running high-quality evaluations easy.\n\nGet started by creating your first evaluation.\nQuickly assess the performance of your application using our off-the-shelf evaluators as a starting point.\nAnalyze results of evaluations in the LangSmith UI and compare results over time.\nEasily collect human feedback on your data to improve your application.\n\nPrompt Engineering\u200b\nWhile traditional software applications are built by writing code, AI applications involve writing prompts to instruct the LLM on what to do. LangSmith provides a set of tools designed to enable and facilitate prompt engineering to help you find the perfect prompt for your applicati

In [12]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model="llama2")

In [13]:
## Retrieval Chain, Document Chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
Answer the following question based on the provided context:
<context>
{context}
</context>

"""

)

documents_chain = create_stuff_documents_chain(llm,prompt)
documents_chain


RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based on the provided context:\n<context>\n{context}\n</context>\n\n'), additional_kwargs={})])
| ChatOllama(model='llama2')
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])

In [14]:
from langchain_core.documents import Document
documents_chain.invoke({
    "input":"LangSmith has two usage limits: total traces and total runs. What are they?",
    "context": [Document(page_content="LangSmith has two usage limits: total traces and total runs. The total traces limit is the maximum number of traces that can be stored in a project, while the total runs limit is the maximum number of runs that can be executed in a project.")]
})

'According to the provided context, the two usage limits for LangSmith are:\n\n1. Total traces: The maximum number of traces that can be stored in a project.\n2. Total runs: The maximum number of runs that can be executed in a project.'

However, we want the documents to first come from the retriever we just set up. That way, we can use the retriever to dynamically select the most relevant documents and pass those in for a given question.

In [15]:
### input-->raetriever--> vectorstoredb

vectorstoredb

In [16]:
retriever=vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever, documents_chain)
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000176FB118950>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based on the provided context:\n<context>\n{context}\n</context>\n\n'), additional_kwargs={})])
            | ChatOll

In [17]:
### Get The response from the llm
response=retrieval_chain.invoke({"input":"LangSmith has two usage limits: total traces and total runs. What are they?"})
response['answer']

'Based on the provided context, the answer to the question "What is LangSmith?" can be inferred as follows:\n\nLangSmith is a framework-agnostic tool for building and managing AI applications. It provides features such as observability, evaluations, and prompt engineering to help developers build high-quality AI applications. LangSmith can be used with or without LangChain\'s open source frameworks langchain and langgraph. To set up LangSmith tracing, developers can enable it with a single environment variable. Additionally, LangSmith provides off-the-shelf evaluators for assessing the performance of AI applications and collecting human feedback to improve them.'

In [ ]:
response

{'input': 'LangSmith has two usage limits: total traces and total runs. What are they?',
 'context': [Document(id='62ba6171-7e47-4e87-a59a-5a386b01f8be', metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications.', 'language': 'en'}, page_content="LangSmith + LangChain OSSLangSmith is framework-agnostic —\xa0it can be used with or without LangChain's open source frameworks langchain and langgraph.If you are using either of these, you can enable LangSmith tracing with a single environment variable.\nFor more see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.\nObservability\u200b\nObservability is important for any software application, but especially so for LLM applications. LLMs are non-deterministic by nature, meaning they can produce unexpected results. This makes them trickier than normal to de

In [ ]:
response['context']

[Document(id='62ba6171-7e47-4e87-a59a-5a386b01f8be', metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications.', 'language': 'en'}, page_content="LangSmith + LangChain OSSLangSmith is framework-agnostic —\xa0it can be used with or without LangChain's open source frameworks langchain and langgraph.If you are using either of these, you can enable LangSmith tracing with a single environment variable.\nFor more see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.\nObservability\u200b\nObservability is important for any software application, but especially so for LLM applications. LLMs are non-deterministic by nature, meaning they can produce unexpected results. This makes them trickier than normal to debug.\nThis is where LangSmith can help! LangSmith has LLM-native observability, allowing you to get m